In [1]:
import numpy as np

In [2]:
substatList = ["HP%", "HP", "ATK%", "ATK", "DEF%", "DEF", "EM", "ER%", "CR%", "CD%"]
#substatBase = [0.0496, 253.9, 0.0496, 16.54, 0.0620, 19.68, 19.82, 0.0551, 0.0331, 0.0662]
substatBase = [0.058, 299, 0.058, 19, 0.073, 23, 23, 0.065, 0.039, 0.078] #These rolls are perfect, not really the best-case scenario for simulations
ittoATK = 191
ittoDEF = 930
ittoCR = 0.242
ittoCD = 0.50
ittoA2 = 0.35
oppulentDEF = 0.54
oppulentGeo = 0.24
feather = 311
hatCR = 0.311
hatCD = 0.622
sandsDEF = 0.583
cupGeo = 0.466

In [3]:
def artiStats():
    totals = [20]
    nums = []
    for i in totals:
        total = i
        for i in range(2):
            val = np.random.randint(0, 8) #changed to 8 to have 7/7/6 possibilities for each 
            nums.append(val)
            total -= val
        nums.append(total)
    liquidrolls = [2,2,2,2,2+nums[0],2,2,2,2+nums[1],2+nums[2]]
    subStats = {substatList[i]: substatBase[i]*liquidrolls[i] for i in range(len(substatList))}
    return subStats

In [4]:
def calcRedhorn(refine, buffs, atk):
    if refine == "R1":
        redhornBonus = 0.28
        redhornDEF = 0.20
    elif refine == "R5":
        redhornBonus = 0.56
        redhornDEF = 0.40
    if buffs == True:
        geoReso = 0.15
        if atk == True:
            noblesse = 0.20
            totm = 0.20
        else:
            noblesse = 0
            totm = 0
        gorouC6 = 0.40
        gorouDEF = 371
        gorouGeo = 0.15
        gorouA2 = 0.25
        geoShred = -0.15
    else:
        geoReso = 0
        noblesse = 0
        totm = 0
        gorouC6 = 0
        gorouDEF = 0
        gorouGeo = 0
        gorouA2 = 0
        geoShred = 0.1
    artifactStats = artiStats()  
    baseATK = ittoATK+608
    baseDEF = ittoDEF
    artiATK = artifactStats["ATK"]+feather
    totalATKP = artifactStats["ATK%"]+noblesse+totm
    totalATK = artiATK+(baseATK*(1+totalATKP))
    artiDEF = artifactStats["DEF"]
    totalDEFP = artifactStats["DEF%"]+oppulentDEF+sandsDEF+gorouA2+redhornDEF
    totalDEF = artiDEF + (baseDEF*(1+totalDEFP))+gorouDEF
    newATK = (totalDEF*1.0368)+totalATK
    cRIT = artifactStats["CR%"]+ittoCR+hatCR
    cDMG = artifactStats["CD%"]+ittoCD+gorouC6+0.662
    geoDMG = cupGeo+oppulentGeo+gorouGeo
    extraDMG = geoReso
    hit1 = (((newATK*1.2643)+(totalDEF*(redhornBonus)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    hit2 = (((newATK*1.2077)+(totalDEF*(redhornBonus)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    hit3 = (((newATK*1.453)+(totalDEF*(redhornBonus)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    hit4 =(((newATK*1.8398)+(totalDEF*(redhornBonus)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    comboslash = (((newATK*1.802)+(totalDEF*(redhornBonus+ittoA2)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    finalslash = (((newATK*3.774)+(totalDEF*(redhornBonus+ittoA2)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    skill = (((newATK*5.5296))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    rotation = skill + ((hit1+hit2+hit3+hit4)+hit1+hit2)+((4*comboslash)+(2*comboslash)+(2*finalslash))
    dps = rotation//20
    return dps

In [5]:
def calcSerpentSpine(refine, buffs, atk):
    if refine == "R1":
        ssBonus = 0.30
    elif refine == "R5":
        ssBonus = 0.50
    if buffs == True:
        geoReso = 0.15
        if atk == True:
            noblesse = 0.20
            totm = 0.20
        else:
            noblesse = 0
            totm = 0
        gorouC6 = 0.40
        gorouDEF = 371
        gorouGeo = 0.15
        gorouA2 = 0.25
        geoShred = -0.15
    else:
        geoReso = 0
        noblesse = 0
        totm = 0
        gorouC6 = 0
        gorouDEF = 0
        gorouGeo = 0
        gorouA2 = 0
        geoShred = 0.1
    artifactStats = artiStats() 
    baseATK = ittoATK+510
    baseDEF = ittoDEF
    artiATK = artifactStats["ATK"]+feather
    totalATKP = artifactStats["ATK%"]+noblesse+totm
    totalATK = artiATK+(baseATK*(1+totalATKP))
    artiDEF = artifactStats["DEF"]
    totalDEFP = artifactStats["DEF%"]+oppulentDEF+sandsDEF+gorouA2
    totalDEF = artiDEF + (baseDEF*(1+totalDEFP))+gorouDEF
    newATK = (totalDEF*1.0368)+totalATK
    cRIT = artifactStats["CR%"]+ittoCR+0.267
    cDMG = artifactStats["CD%"]+ittoCD+gorouC6+hatCD
    geoDMG = cupGeo+oppulentGeo+gorouGeo
    extraDMG = geoReso+ssBonus
    hit1 = (((newATK*1.2643)+(totalDEF*(0)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    hit2 = (((newATK*1.2077)+(totalDEF*(0)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    hit3 = (((newATK*1.453)+(totalDEF*(0)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    hit4 =(((newATK*1.8398)+(totalDEF*(0)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    comboslash = (((newATK*1.802)+(totalDEF*(0+ittoA2)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    finalslash = (((newATK*3.774)+(totalDEF*(0+ittoA2)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    skill = (((newATK*5.5296))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    rotation = skill + ((hit1+hit2+hit3+hit4)+hit1+hit2)+((4*comboslash)+(2*comboslash)+(2*finalslash))
    dps = rotation//20
    return dps

In [6]:
def calcWhiteblind(refine, buffs, atk):
    if refine == "R1":
        wbATK = 0.24
        wbDEF = 0.24
    elif refine == "R5":
        wbATK = 0.48
        wbDEF = 0.48
    if buffs == True:
        geoReso = 0.15
        if atk == True:
            noblesse = 0.20
            totm = 0.20
        else:
            noblesse = 0
            totm = 0
        gorouC6 = 0.40
        gorouDEF = 371
        gorouGeo = 0.15
        gorouA2 = 0.25
        geoShred = -0.15
    else:
        geoReso = 0
        noblesse = 0
        totm = 0
        gorouC6 = 0
        gorouDEF = 0
        gorouGeo = 0
        gorouA2 = 0
        geoShred = 0.1
    artifactStats = artiStats() 
    baseATK = ittoATK+510
    baseDEF = ittoDEF
    artiATK = artifactStats["ATK"]+feather
    totalATKP = artifactStats["ATK%"]+noblesse+totm+wbATK
    totalATK = artiATK+(baseATK*(1+totalATKP))
    artiDEF = artifactStats["DEF"]
    totalDEFP = artifactStats["DEF%"]+oppulentDEF+sandsDEF+gorouA2+wbDEF+0.517
    totalDEF = artiDEF + (baseDEF*(1+totalDEFP))+gorouDEF
    newATK = (totalDEF*1.0368)+totalATK
    if artifactStats["CR%"] < 0.15:
        cRIT = artifactStats["CR%"]+ittoCR+hatCR
        cDMG = artifactStats["CD%"]+ittoCD+gorouC6
    else:
        cRIT = artifactStats["CR%"]+ittoCR
        cDMG = artifactStats["CD%"]+ittoCD+gorouC6+hatCD
    geoDMG = cupGeo+oppulentGeo+gorouGeo
    extraDMG = geoReso
    hit1 = (((newATK*1.2643)+(totalDEF*(0)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    hit2 = (((newATK*1.2077)+(totalDEF*(0)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    hit3 = (((newATK*1.453)+(totalDEF*(0)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    hit4 =(((newATK*1.8398)+(totalDEF*(0)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    comboslash = (((newATK*1.802)+(totalDEF*(0+ittoA2)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    finalslash = (((newATK*3.774)+(totalDEF*(0+ittoA2)))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    skill = (((newATK*5.5296))*(1+(geoDMG+extraDMG))*(1+(cDMG*cRIT)))*(0.487*(1-geoShred))
    rotation = skill + ((hit1+hit2+hit3+hit4)+hit1+hit2)+((4*comboslash)+(2*comboslash)+(2*finalslash))
    dps = rotation//20
    return dps

<b>Rotation</b>: Q -> N4 -> SCA5D -> E -> N2SCA3 (as provided by Jrm.Spirit)
<ul> 
    <li>20 fixed subs, 20 liquid subs optimized for <i>DEF%, CR%,</i> and <i>CD%</i>. (all perfect rolls)</li>
    <li>C0 Lvl 90 Itto 10/10/10 talents against Lvl 100 enemy with 10% universal resistance</li>
    <li> 4p Opulent, max stacks, DEF%/Geo%/CR or CD
    <li> Buffs include NO, ToTM, , C6 Gorou E at Lvl 10, Zhongli Shred, Geo Reso</li>
    </ul>
    

In [7]:
simul = 1000
isBuffed = False
isNOTotM = False
SerpentSpine = []
for i in range(simul):
    totaldps = calcSerpentSpine("R5", isBuffed, isNOTotM)
    SerpentSpine.append(totaldps)
print(f"Average DPS (20s) using Serpent Spine R1 is {np.average(SerpentSpine)} with standard deviation of {np.std(SerpentSpine)}")

Average DPS (20s) using Serpent Spine R1 is 15626.176 with standard deviation of 468.3573433010312


In [8]:
Whiteblind = []
for i in range(simul):
    totaldps = calcWhiteblind("R5", isBuffed, isNOTotM)
    Whiteblind.append(totaldps)
print(f"Average DPS (20s) using Whiteblind R5 is {np.average(Whiteblind)} with standard deviation of {np.std(Whiteblind)}")

Average DPS (20s) using Whiteblind R5 is 13240.778 with standard deviation of 612.423777066175
